In [1]:
!pip install sentence_transformers
!pip -q install langchain tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers
!pip -q install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.6 MB/s eta 0:00:0

In [2]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [3]:
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip

--2024-11-19 03:30:03--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2024-11-19 03:30:03--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2024-11-19 03:30:03 (66.0 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638

In [4]:
# Initializes a DirectoryLoader to load text files from the specified directory.
loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)

In [5]:
# Loads all the documents present in the directory.
documents = loader.load()

In [6]:
documents

[Document(metadata={'source': 'stock_market_june_2023/Kiplin Metals Inc.txt'}, page_content='The global shift towards electric vehicles (EVs) is driving an exponential increase in electricity demand. Countries worldwide, including oil-producing nations, are turning to nuclear energy to meet this growing need while transitioning away from fossil fuels. Consequently, the demand for uranium, essential for fueling the nuclear power sector, has experienced a significant surge in the past year after decades of stagnation, clearly showcased in the uranium spot price.\n\nEveryone is drilling for uranium, and companies like Kiplin Metals Inc. (TSXV: KIP | FWB: 17G1 | OTC: ALDVF), which are strategically located in uranium-rich regions like Saskatchewan, Canada, stand to gain the most.\nKiplin’s mining location next to Cluff Lake in Saskatchewan is a known vein of uranium that attracted companies like Urano in the past. The area has historically produced approximately 62,000,000 lbs of yellowcak

In [7]:
# Initializes a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Splits the loaded documents into chunks of text using the defined text_splitter.
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

49

In [9]:
texts[2]

Document(metadata={'source': 'stock_market_june_2023/Kiplin Metals Inc.txt'}, page_content='Kiplin’s mining location next to Cluff Lake in Saskatchewan is a known vein of uranium that attracted companies like Urano in the past. The area has historically produced approximately 62,000,000 lbs of yellowcake uranium, making it a competitive place for other companies to buy claims.')

In [10]:
#Falcon 7b model
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_new_tokens=1024,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [11]:
#Embeddings
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
llm = HuggingFacePipeline(pipeline=pipeline)

model_name = "intfloat/e5-large-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)

<ipython-input-12-8f75202eb7f1>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)
<ipython-input-12-8f75202eb7f1>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [13]:
#Db
# Sets the directory where the embeddings will be stored.
persist_directory = 'db'

# Sets the HuggingFaceEmbeddings object as the embedding to use.
embedding = hf

# Uses the Chroma module to convert the texts into embeddings using the specified HuggingFace embeddings.
# The resulting embeddings are stored in the persist_directory.
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=hf,
                                 persist_directory=persist_directory)

In [14]:
# Persists the generated embeddings database to disk.
vectordb.persist()

# Releases the memory held by the vectordb object by setting it to None.
vectordb = None

 #Re-initializes the Chroma object from the persisted directory with the specified HuggingFace embeddings.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=hf)

<ipython-input-14-dda3a63f1ef4>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
<ipython-input-14-dda3a63f1ef4>:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


In [15]:


# Initializes a retriever object from the vectordb with a specified number of search results (k=3).
# The vectordb.as_retriever() function is likely converting the Chroma object, which contains vectorized representations of text, into a retriever object.
# The retriever can be used to find the most similar vectors in the database given a query vector.
# The search_kwargs={"k": 3} argument suggests that the retriever will return the top 3 most similar vectors for each query.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# create the chain to answer questions
# Initializes a RetrievalQA object with the specified HuggingFacePipeline (llm), retriever, and chain type.
# The return_source_documents parameter set to True means the original source documents will be included in the returned results.
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)


 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])

In [16]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

<ipython-input-16-30e823e62489>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_response = qa_chain(query)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Arbor Metals Corp.
Tesla Inc.
Kiplin Metals Inc.
Microsoft
NVIDIA
Shopify
Intuitive Surgical
MercadoLibre
Growth stocks like these are ideal for investors seeking short-term gains and long-term resilience. These companies are primed to withstand future market fluctuations through innovation, market opportunities, embedding in recession-resistant sectors, and committing to diversifying their portfolios.

Future-proofing your portfolio with growth stocks isn’t just about singling out products and industries that will always be in demand. 
It’s also about identifying companies engineered from inception to adapt to industry shifts. Microsoft (NASDAQ: MSFT) fits these criteria like a glove.

Institutional ETFs assembled by Goldman Sachs and Vanguard rely on growth stocks to drive their price in all market conditions, and your por

In [17]:
query = "How much has Microsoft invested in OpenAI?"
llm_response = qa_chain(query)
llm_response

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'query': 'How much has Microsoft invested in OpenAI?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nLet’s focus on Microsoft’s artificial intelligence (AI) prospects. As AI propels itself into the mainstream, it’s not surprising to see investors eager to fortify their portfolios with AI stocks. Microsoft’s recent $1 billion investment in OpenAI – the creator of ChatGPT and the AI image generator Dall-E- positions the stock as a prime candidate for investment.\n\nMicrosoft’s strategic move to integrate OpenAI’s products within its platform positions the company as a preferred platform for organizations looking to develop AI models or incorporate them into their business operations. Microsoft’s focus on building AI tools as platforms is a savvy strategic decision that will likely broaden its already considerable enterprise customer base and service suite.\

In [22]:
query = "What were the general reasons of the flight delay?"
llm_response1 = qa_chain(query)
process_llm_response(llm_response1)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The industry faced significant challenges in recent years. The COVID-19 pandemic led to the cancellation and postponement of many surgical procedures. Resurgences in cases, particularly in Asia, continue to impact procedure volumes. Inflation and a shortage of healthcare workers have also raised operational costs for hospitals, affecting the adoption of surgical systems.

The global shift towards electric vehicles (EVs) is driving an exponential increase in electricity demand. Countries worldwide, including oil-producing nations, are turning to nuclear energy to meet this growing need while transitioning away from fossil fuels. Consequently, the demand for uranium, essential for fueling the nuclear power sector, has experienced a significant surge in the past year after decades of stagnation, clearly showcased in the uranium